# LangChain Expression Language (LCEL)

### Initialize model

In [7]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=1)

In [9]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# Create a prompt template for the chat model to give the response of all message in a funny way

example = [
    HumanMessage(
        content="What is the capital of France?"
    ),
    AIMessage(
        content="The capital of France is Paris, but I prefer to call it 'The City of Lights and Croissants!'"
    ),
    HumanMessage(
        content="What is the capital of Germany?"
    ),
    AIMessage(content="The capital of Germany is Berlin, but I like to think of it as 'The Land of Sausages and Beer!'"
              ),
    HumanMessage(
        content="What is the capital of Italy?"
    ),
    AIMessage(
        content="The capital of Italy is Rome, but I prefer to call it 'The Eternal City of Pizza and Pasta!'"
    ),
    HumanMessage(
        content="What is the capital of Spain?"
    ),
    AIMessage(
        content="The capital of Spain is Madrid, but I like to think of it as 'The Land of Flamenco and Tapas!'"
    ),
]

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are a helpful assistant that gives funny responses to all messages."
        ),
        *example,
        MessagesPlaceholder("input")
    ]
)

## RunnableSequence

In [16]:
from langchain_core.runnables import RunnableSequence, RunnableLambda


addition = RunnableLambda(lambda x: x + 10)
multiplication = RunnableLambda(lambda x: x * 2)

chain = RunnableSequence(addition, multiplication)

chain.invoke(10)

40

## RunnableParallel

In [1]:
from langchain_core.runnables import RunnableParallel, RunnableLambda


addition = RunnableLambda(lambda x: x + 10)
multiplication = RunnableLambda(lambda x: x * 2)

chain = RunnableParallel({
    "add": addition,
    "mul": multiplication
})

chain.invoke(10)

{'add': 20, 'mul': 20}

## Shorthand method or run these composition

### The | Operator 

In [26]:
# We know that both prompt and llm are build using runnable interface

chain = prompt | llm

chain.invoke({"input": [HumanMessage(content="what is the capital of india")]})

AIMessage(content="The capital of India is New Delhi, but I like to call it 'The City of Spicy Curries and Traffic Jams!'", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 184, 'total_tokens': 212, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'finish_reason': 'stop', 'logprobs': None}, id='run-c143f350-9233-4df2-871f-7f224ff741a8-0', usage_metadata={'input_tokens': 184, 'output_tokens': 28, 'total_tokens': 212, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### The `.pipe()` method

In [33]:
chain = prompt.pipe(llm)

chain.invoke({"input": [HumanMessage(content="what is the capital of india")]})

AIMessage(content="The capital of India is New Delhi! But I like to refer to it as 'The City of Curries and Chaos!'", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 184, 'total_tokens': 210, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_dbaca60df0', 'finish_reason': 'stop', 'logprobs': None}, id='run-a8a91dc7-5da1-499f-8289-cde957dfabdf-0', usage_metadata={'input_tokens': 184, 'output_tokens': 26, 'total_tokens': 210, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Examples of `LCEL`

#### 1. How to run custom functions

- This is important when you want to pass arbitrary function as [Runnables](./runnable.md).
- When you want to build a functionality, not provided by the LangChain. So custom functions are used as [RunnableLambda](./runnable.md).
- **💀 Note:** All these functions only accept **"SINGLE"** arguments. To pass multiple you have to build wrapper that single dict as input and unpack them into multiple.

### 1. Using Constructor

- We wrap our custom logic using the **`RunnableLambda`** constructor.

In [75]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda


def get_len(text: str):
    return len(text)


def _multiple_len_function(text1, text2):
    return len(text1) * len(text2)


def multiply_len_function(_dict):
    return _multiple_len_function(_dict["text1"], _dict["text2"])


def print_dict(_dict):
    print(_dict)


prompt = ChatPromptTemplate.from_template("what is {a} + {b}")

chain = (
    {
        "a": itemgetter("foo") | RunnableLambda(get_len),
        "b": {
            "text1": itemgetter("foo"), "text2": itemgetter("bar")
        } | RunnableLambda(multiply_len_function)
    }
    | prompt | llm
)

chain2 = (
    {
        "a": {
            "text1": itemgetter("a"),  # 3
            "text2": itemgetter("b")  # 3 ==> 3 * 3 = 9
        } | RunnableLambda(multiply_len_function),
        "b": itemgetter("b") | RunnableLambda(lambda x: len(x))
    } | RunnableLambda(print_dict)
)


chain2.invoke({"a": "bob", "b": "pop"})

print(chain.invoke({"foo": "abdurrab", "bar": "khan"}))


def hello(a: int):
    print("A ", a)


chain5 = RunnableLambda(hello)

# chain5.invoke(2, 5) ⚠️ We can't pass more than one arguments, but dict, list can.


def get_dict(a):
    print(a)


chain6 = RunnableLambda(get_dict)

chain6.invoke([1, 2, 3, 4])
chain6.invoke({"a": 1, "b": 2})

{'a': 9, 'b': 3}
content='8 + 32 equals 40.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 14, 'total_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'finish_reason': 'stop', 'logprobs': None} id='run-40da9d88-f2cc-4cd6-b6d3-2923458938a1-0' usage_metadata={'input_tokens': 14, 'output_tokens': 9, 'total_tokens': 23, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
[1, 2, 3, 4]
{'a': 1, 'b': 2}


### The convenience `@chain` decorator

- You convert arbitrary function into the chain by using **`@chain`** decorators.
- This functionality is equivalent to wrapping the function in **`RunnableLambda`** constructor as shown above.